In [1]:
import ray
import gymnasium as gym

In [2]:
ray.init()

2023-04-20 14:53:47,602	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.10
Ray version:,2.3.1


In [40]:
@ray.remote
class CustomEnv(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.env = env
    
    def step(self, action):
        super().step(action)
    

In [41]:
envs = [gym.make("CartPole-v1") for _ in range(4)]

In [42]:
envs = [CustomEnv.remote(env) for env in envs]

In [43]:
reset_handle = [env.reset.remote() for env in envs]

In [50]:
ray.get(envs[0].action_space.remote)

ValueError: 'object_refs' must either be an ObjectRef or a list of ObjectRefs.

In [47]:
while True:
    actions = [ray.get(env.action_space.remote()).sample() for env in envs]
    step_handles = [env.step.remote(action) for env, action in zip(envs, actions)]
    dones = ray.get([handle[2] for handle in step_handles])
    break

RayTaskError(TypeError): [36mray::CustomEnv.action_space()[39m (pid=24192, ip=127.0.0.1, repr=<CustomEnv<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>>)
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 803, in ray._raylet.execute_task.function_executor
  File "c:\Users\tomkj\miniconda3\envs\rl\lib\site-packages\ray\_private\function_manager.py", line 674, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\tomkj\miniconda3\envs\rl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 466, in _resume_span
    return method(self, *_args, **_kwargs)
  File "C:\Users\tomkj\AppData\Local\Temp\ipykernel_32976\742329856.py", line 11, in action_space
TypeError: 'Discrete' object is not callable

In [16]:
env

ObjectRef(8849b62d89cb30f9ffffffffffffffffffffffff0100000001000000)

In [17]:
env = ray.get(env)

In [19]:
env.reset()

(array([-0.03055554,  0.00070518,  0.01814227, -0.02808008], dtype=float32),
 {})

In [27]:
@ray.remote
def make_env():
    return gym.make("CartPole-v1")

In [28]:
env = make_env.remote()

In [29]:
env.reset.remote()

AttributeError: 'ray._raylet.ObjectRef' object has no attribute 'reset'